In [ ]:
import cv2
import numpy as np


# Function to process the video
def process_local_video(video_path):
    # Read the video using OpenCV VideoCapture
    cap = cv2.VideoCapture(video_path)

    # Process the video frame by frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect lanes
        lanes_frame = detect_lanes(frame)

        # Display the resulting frame
        cv2.imshow('Lanes Detection', lanes_frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture
    cap.release()
    cv2.destroyAllWindows()


# Function to detect lanes in a frame
def detect_lanes(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blur, 50, 150)

    # Define region of interest (ROI)
    height, width = frame.shape[:2]
    mask = np.zeros_like(edges)
    roi = np.array([[(0, height), (width // 2, height // 2), (width, height)]], dtype=np.int32)
    cv2.fillPoly(mask, roi, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough Transform to detect lines
    lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    # Draw lines on a blank image
    line_image = np.zeros_like(frame)
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # Combine detected lines with original frame
    lanes_image = cv2.addWeighted(frame, 0.8, line_image, 1, 0)

    return lanes_image


# Main function
def main():
    # Input local video file path
    video_path = "/Users/ruksarchaudhary/Desktop/project/Face_Recognition/.venv/faces/lanes_clip.mp4"

    # Process the local video
    process_local_video(video_path)


if __name__ == "__main__":
    main()
